# Social Listening Data Analytics & Visualization

This notebook provides comprehensive visual analysis of social media listening data exported from BigQuery.

## Features:
- **Time Series Analysis**: Evolution of engagement metrics over time
- **Platform Comparison**: Cross-platform performance analysis
- **Interactive Dashboards**: Dynamic visualizations with Plotly
- **Multiple Chart Types**: Line charts, bar charts, heatmaps, and more
- **Professional Styling**: Technical and polished appearance

## Prerequisites:
- Run the data extraction notebook first to generate CSV files
- Ensure all visualization libraries are installed via `uv sync`

## 1. Environment Setup & Library Imports

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pyo.init_notebook_mode(connected=True)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✓ All libraries imported successfully")
print("✓ Visualization environment ready")

## 2. Data Loading & Preprocessing

In [ ]:
# Find and load the most recent aggregated CSV file
data_dir = 'data'
aggregated_files = glob.glob(os.path.join(data_dir, 'social_listening_aggregated_topic238_*.csv'))

if not aggregated_files:
    print("❌ No aggregated CSV files found in data directory.")
    print("   Please run the data extraction notebook first.")
else:
    # Get the most recent file
    latest_file = max(aggregated_files, key=os.path.getctime)
    print(f"📊 Loading data from: {latest_file}")
    
    # Load the data
    df = pd.read_csv(latest_file)
    
    # Data preprocessing
    df['fecha'] = pd.to_datetime(df['fecha'])
    df = df.sort_values('fecha')
    
    # Fill NaN values with 0 for numeric columns
    numeric_cols = ['posts', 'likes', 'comments', 'shares', 'engagement', 'views']
    df[numeric_cols] = df[numeric_cols].fillna(0)
    
    print(f"✓ Data loaded successfully")
    print(f"✓ Shape: {df.shape}")
    print(f"✓ Date range: {df['fecha'].min().strftime('%Y-%m-%d')} to {df['fecha'].max().strftime('%Y-%m-%d')}")
    print(f"✓ Platforms: {', '.join(df['platform'].unique())}")
    
    # Display basic statistics
    print("\n📈 Dataset Overview:")
    display(df.head())
    print("\n📊 Summary Statistics:")
    display(df.describe())

## 3. Time Series Evolution Analysis

### 3.1 Interactive Time Series Dashboard

In [ ]:
# Create comprehensive time series dashboard
def create_time_series_dashboard(data):
    """
    Creates an interactive dashboard showing evolution of key metrics over time
    """
    # Aggregate data by date
    daily_agg = data.groupby('fecha').agg({
        'posts': 'sum',
        'likes': 'sum', 
        'comments': 'sum',
        'shares': 'sum',
        'engagement': 'sum',
        'views': 'sum'
    }).reset_index()
    
    # Create subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Daily Posts Volume', 'Total Engagement Evolution',
            'Likes & Comments Trend', 'Shares & Views Analysis',
            'Engagement Rate Analysis', 'Platform Activity Distribution'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": True}],
               [{"secondary_y": True}, {"secondary_y": True}],
               [{"secondary_y": False}, {"type": "bar"}]]
    )
    
    # 1. Daily Posts Volume
    fig.add_trace(
        go.Scatter(
            x=daily_agg['fecha'], 
            y=daily_agg['posts'],
            mode='lines+markers',
            name='Daily Posts',
            line=dict(color='#1f77b4', width=3),
            marker=dict(size=8)
        ),
        row=1, col=1
    )
    
    # 2. Total Engagement Evolution
    fig.add_trace(
        go.Scatter(
            x=daily_agg['fecha'], 
            y=daily_agg['engagement'],
            mode='lines+markers',
            name='Total Engagement',
            line=dict(color='#ff7f0e', width=3),
            marker=dict(size=8)
        ),
        row=1, col=2
    )
    
    # Add engagement rate on secondary y-axis
    daily_agg['engagement_rate'] = (daily_agg['engagement'] / daily_agg['posts']).fillna(0)
    fig.add_trace(
        go.Scatter(
            x=daily_agg['fecha'], 
            y=daily_agg['engagement_rate'],
            mode='lines',
            name='Engagement Rate',
            line=dict(color='#d62728', width=2, dash='dash'),
            yaxis='y2'
        ),
        row=1, col=2, secondary_y=True
    )
    
    # 3. Likes & Comments Trend
    fig.add_trace(
        go.Scatter(
            x=daily_agg['fecha'], 
            y=daily_agg['likes'],
            mode='lines+markers',
            name='Likes',
            line=dict(color='#2ca02c', width=3)
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=daily_agg['fecha'], 
            y=daily_agg['comments'],
            mode='lines+markers',
            name='Comments',
            line=dict(color='#9467bd', width=3),
            yaxis='y4'
        ),
        row=2, col=1, secondary_y=True
    )
    
    # 4. Shares & Views Analysis
    fig.add_trace(
        go.Scatter(
            x=daily_agg['fecha'], 
            y=daily_agg['shares'],
            mode='lines+markers',
            name='Shares',
            line=dict(color='#8c564b', width=3)
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=daily_agg['fecha'], 
            y=daily_agg['views'],
            mode='lines+markers',
            name='Views',
            line=dict(color='#e377c2', width=3),
            yaxis='y6'
        ),
        row=2, col=2, secondary_y=True
    )
    
    # 5. Engagement Rate Analysis (Area Chart)
    fig.add_trace(
        go.Scatter(
            x=daily_agg['fecha'], 
            y=daily_agg['engagement_rate'],
            mode='lines',
            name='Avg Engagement Rate',
            fill='tonexty',
            line=dict(color='#17becf', width=2)
        ),
        row=3, col=1
    )
    
    # 6. Platform Activity Distribution
    platform_totals = data.groupby('platform')['posts'].sum().reset_index()
    fig.add_trace(
        go.Bar(
            x=platform_totals['platform'],
            y=platform_totals['posts'],
            name='Posts by Platform',
            marker_color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
        ),
        row=3, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=1200,
        title_text="<b>Social Listening Analytics Dashboard</b><br><sub>Topic 238 - Comprehensive Metrics Evolution</sub>",
        title_x=0.5,
        title_font_size=20,
        showlegend=True,
        template="plotly_white",
        font=dict(size=12)
    )
    
    # Update x-axes
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    
    return fig

# Create and display the dashboard
dashboard_fig = create_time_series_dashboard(df)
dashboard_fig.show()

### 3.2 Advanced Time Series Analysis with Moving Averages

In [ ]:
# Create advanced time series with trend analysis
def create_trend_analysis(data):
    """
    Creates sophisticated trend analysis with moving averages and seasonality
    """
    # Aggregate daily data
    daily_data = data.groupby('fecha').agg({
        'posts': 'sum',
        'engagement': 'sum',
        'likes': 'sum',
        'comments': 'sum'
    }).reset_index()
    
    # Calculate moving averages
    daily_data['posts_ma7'] = daily_data['posts'].rolling(window=7, center=True).mean()
    daily_data['engagement_ma7'] = daily_data['engagement'].rolling(window=7, center=True).mean()
    
    # Create the figure
    fig = go.Figure()
    
    # Raw data
    fig.add_trace(go.Scatter(
        x=daily_data['fecha'],
        y=daily_data['posts'],
        mode='markers+lines',
        name='Daily Posts (Raw)',
        line=dict(color='lightblue', width=1),
        marker=dict(size=4, opacity=0.6)
    ))
    
    # Moving average
    fig.add_trace(go.Scatter(
        x=daily_data['fecha'],
        y=daily_data['posts_ma7'],
        mode='lines',
        name='7-Day Moving Average',
        line=dict(color='darkblue', width=3)
    ))
    
    # Add confidence interval
    posts_std = daily_data['posts'].std()
    upper_bound = daily_data['posts_ma7'] + posts_std
    lower_bound = daily_data['posts_ma7'] - posts_std
    
    fig.add_trace(go.Scatter(
        x=daily_data['fecha'],
        y=upper_bound,
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ))
    
    fig.add_trace(go.Scatter(
        x=daily_data['fecha'],
        y=lower_bound,
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(0,100,80,0.1)',
        name='Confidence Interval',
        hoverinfo='skip'
    ))
    
    fig.update_layout(
        title="<b>Social Media Activity Trend Analysis</b><br><sub>Posts Volume with Statistical Indicators</sub>",
        title_x=0.5,
        xaxis_title="Date",
        yaxis_title="Number of Posts",
        template="plotly_white",
        height=500,
        hovermode='x unified'
    )
    
    return fig

# Display trend analysis
trend_fig = create_trend_analysis(df)
trend_fig.show()

## 4. Platform Comparison & Performance Analysis

### 4.1 Multi-Dimensional Platform Comparison

In [ ]:
# Create comprehensive platform comparison
def create_platform_analysis(data):
    """
    Multi-faceted platform performance analysis
    """
    # Platform aggregation
    platform_stats = data.groupby('platform').agg({
        'posts': 'sum',
        'likes': 'sum',
        'comments': 'sum', 
        'shares': 'sum',
        'engagement': 'sum',
        'views': 'sum'
    }).reset_index()
    
    # Calculate derived metrics
    platform_stats['avg_engagement_per_post'] = platform_stats['engagement'] / platform_stats['posts']
    platform_stats['likes_per_post'] = platform_stats['likes'] / platform_stats['posts']
    platform_stats['comments_per_post'] = platform_stats['comments'] / platform_stats['posts']
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Total Activity by Platform',
            'Engagement Efficiency',
            'Platform Share Distribution',
            'Performance Radar Chart'
        ),
        specs=[[{"type": "bar"}, {"type": "scatter"}],
               [{"type": "pie"}, {"type": "scatterpolar"}]]
    )
    
    # 1. Total Activity (Stacked Bar)
    fig.add_trace(
        go.Bar(
            x=platform_stats['platform'],
            y=platform_stats['posts'],
            name='Posts',
            marker_color='#1f77b4'
        ),
        row=1, col=1
    )
    
    # 2. Engagement Efficiency (Bubble Chart)
    fig.add_trace(
        go.Scatter(
            x=platform_stats['posts'],
            y=platform_stats['avg_engagement_per_post'],
            mode='markers+text',
            text=platform_stats['platform'],
            textposition="middle center",
            marker=dict(
                size=platform_stats['engagement']/100,
                color=platform_stats['engagement'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Total Engagement")
            ),
            name='Efficiency'
        ),
        row=1, col=2
    )
    
    # 3. Platform Share (Pie Chart)
    fig.add_trace(
        go.Pie(
            labels=platform_stats['platform'],
            values=platform_stats['posts'],
            name="Platform Share",
            marker=dict(colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'])
        ),
        row=2, col=1
    )
    
    # 4. Performance Radar Chart
    for i, platform in enumerate(platform_stats['platform']):
        platform_data = platform_stats[platform_stats['platform'] == platform]
        
        # Normalize metrics (0-100 scale)
        metrics = {
            'Posts': (platform_data['posts'].iloc[0] / platform_stats['posts'].max()) * 100,
            'Avg Engagement': (platform_data['avg_engagement_per_post'].iloc[0] / platform_stats['avg_engagement_per_post'].max()) * 100,
            'Likes/Post': (platform_data['likes_per_post'].iloc[0] / platform_stats['likes_per_post'].max()) * 100,
            'Comments/Post': (platform_data['comments_per_post'].iloc[0] / platform_stats['comments_per_post'].max()) * 100
        }
        
        fig.add_trace(
            go.Scatterpolar(
                r=list(metrics.values()),
                theta=list(metrics.keys()),
                fill='toself',
                name=platform,
                opacity=0.7
            ),
            row=2, col=2
        )
    
    fig.update_layout(
        height=800,
        title_text="<b>Platform Performance Analysis</b><br><sub>Comprehensive Multi-Metric Comparison</sub>",
        title_x=0.5,
        showlegend=True,
        template="plotly_white"
    )
    
    return fig, platform_stats

# Create and display platform analysis
platform_fig, platform_data = create_platform_analysis(df)
platform_fig.show()

print("\n📊 Platform Performance Summary:")
display(platform_data.round(2))

### 4.2 Time-Based Platform Evolution

In [ ]:
# Platform evolution over time
def create_platform_evolution(data):
    """
    Shows how different platforms perform over time
    """
    # Create stacked area chart
    fig = px.area(
        data, 
        x='fecha', 
        y='posts', 
        color='platform',
        title="<b>Platform Activity Evolution</b><br><sub>Daily Posts Distribution by Platform</sub>",
        labels={
            'fecha': 'Date',
            'posts': 'Number of Posts',
            'platform': 'Platform'
        }
    )
    
    fig.update_layout(
        height=500,
        template="plotly_white",
        title_x=0.5,
        hovermode='x unified'
    )
    
    return fig

# Display platform evolution
evolution_fig = create_platform_evolution(df)
evolution_fig.show()

## 5. Engagement Analysis & Correlation Matrix

In [ ]:
# Advanced engagement analysis
def create_engagement_analysis(data):
    """
    Deep dive into engagement patterns and correlations
    """
    # Prepare correlation data
    corr_data = data[['posts', 'likes', 'comments', 'shares', 'engagement', 'views']].corr()
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Engagement Metrics Correlation Heatmap',
            'Posts vs Engagement Scatter',
            'Engagement Distribution by Platform',
            'Weekly Engagement Patterns'
        ),
        specs=[[{"type": "heatmap"}, {"type": "scatter"}],
               [{"type": "box"}, {"type": "bar"}]]
    )
    
    # 1. Correlation Heatmap
    fig.add_trace(
        go.Heatmap(
            z=corr_data.values,
            x=corr_data.columns,
            y=corr_data.index,
            colorscale='RdBu_r',
            zmid=0,
            text=np.round(corr_data.values, 2),
            texttemplate="%{text}",
            textfont={"size": 10}
        ),
        row=1, col=1
    )
    
    # 2. Posts vs Engagement Scatter (by platform)
    for platform in data['platform'].unique():
        platform_data = data[data['platform'] == platform]
        fig.add_trace(
            go.Scatter(
                x=platform_data['posts'],
                y=platform_data['engagement'],
                mode='markers',
                name=f'{platform}',
                marker=dict(size=8, opacity=0.7)
            ),
            row=1, col=2
        )
    
    # 3. Engagement Distribution Box Plot
    for platform in data['platform'].unique():
        platform_data = data[data['platform'] == platform]
        fig.add_trace(
            go.Box(
                y=platform_data['engagement'],
                name=platform,
                boxpoints='outliers'
            ),
            row=2, col=1
        )
    
    # 4. Weekly Patterns
    data_copy = data.copy()
    data_copy['weekday'] = data_copy['fecha'].dt.day_name()
    weekly_engagement = data_copy.groupby('weekday')['engagement'].mean().reindex([
        'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
    ])
    
    fig.add_trace(
        go.Bar(
            x=weekly_engagement.index,
            y=weekly_engagement.values,
            marker_color='skyblue',
            name='Avg Weekly Engagement'
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        height=800,
        title_text="<b>Engagement Analysis Dashboard</b><br><sub>Patterns, Correlations & Distribution</sub>",
        title_x=0.5,
        showlegend=True,
        template="plotly_white"
    )
    
    return fig

# Create and display engagement analysis
engagement_fig = create_engagement_analysis(df)
engagement_fig.show()

## 6. Advanced Statistical Visualizations

### 6.1 3D Performance Surface Plot

In [ ]:
# Create 3D surface plot for advanced visualization
def create_3d_performance_surface(data):
    """
    Creates a 3D surface showing relationship between posts, engagement, and time
    """
    # Prepare data for 3D visualization
    data_pivot = data.pivot_table(
        index='fecha', 
        columns='platform', 
        values='engagement', 
        fill_value=0
    )
    
    # Create 3D surface
    fig = go.Figure()
    
    # Add surface for each platform
    for i, platform in enumerate(data_pivot.columns):
        fig.add_trace(
            go.Scatter3d(
                x=data_pivot.index,
                y=[platform] * len(data_pivot.index),
                z=data_pivot[platform],
                mode='markers+lines',
                name=platform,
                marker=dict(size=5),
                line=dict(width=3)
            )
        )
    
    fig.update_layout(
        title="<b>3D Engagement Performance Landscape</b>",
        scene=dict(
            xaxis_title="Date",
            yaxis_title="Platform",
            zaxis_title="Engagement Level",
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
        ),
        height=600,
        template="plotly_white"
    )
    
    return fig

# Display 3D visualization
surface_fig = create_3d_performance_surface(df)
surface_fig.show()

### 6.2 Interactive Sunburst Chart

In [ ]:
# Create hierarchical sunburst visualization
def create_sunburst_analysis(data):
    """
    Creates a sunburst chart showing hierarchical breakdown of engagement
    """
    # Prepare hierarchical data
    data_copy = data.copy()
    data_copy['month'] = data_copy['fecha'].dt.strftime('%Y-%m')
    data_copy['week'] = data_copy['fecha'].dt.strftime('%Y-W%U')
    
    # Aggregate data for sunburst
    sunburst_data = data_copy.groupby(['month', 'platform']).agg({
        'engagement': 'sum',
        'posts': 'sum'
    }).reset_index()
    
    fig = go.Figure(go.Sunburst(
        labels=sunburst_data['platform'].tolist() + sunburst_data['month'].unique().tolist(),
        parents=[''] * len(sunburst_data['platform'].unique()) + [''] * len(sunburst_data['month'].unique()),
        values=sunburst_data['engagement'].tolist() + [sunburst_data[sunburst_data['month'] == month]['engagement'].sum() for month in sunburst_data['month'].unique()],
        branchvalues="total",
        hovertemplate='<b>%{label}</b><br>Engagement: %{value}<br>Percentage: %{percentParent}<extra></extra>',
        maxdepth=3
    ))
    
    fig.update_layout(
        title="<b>Hierarchical Engagement Distribution</b><br><sub>Platform and Time Breakdown</sub>",
        title_x=0.5,
        font_size=12,
        height=600
    )
    
    return fig

# Note: Simplified version due to data structure
# Create platform engagement pie chart as alternative
platform_engagement = df.groupby('platform')['engagement'].sum().reset_index()

pie_fig = go.Figure(data=[
    go.Pie(
        labels=platform_engagement['platform'],
        values=platform_engagement['engagement'],
        hole=.3,
        hovertemplate='<b>%{label}</b><br>Engagement: %{value}<br>Share: %{percent}<extra></extra>'
    )
])

pie_fig.update_layout(
    title="<b>Platform Engagement Share</b><br><sub>Total Engagement Distribution</sub>",
    title_x=0.5,
    height=500,
    annotations=[dict(text='Total<br>Engagement', x=0.5, y=0.5, font_size=16, showarrow=False)]
)

pie_fig.show()

## 7. Executive Summary & Key Insights

In [ ]:
# Generate executive summary with key metrics
def generate_executive_summary(data):
    """
    Creates an executive summary with key insights and metrics
    """
    # Calculate key metrics
    total_posts = data['posts'].sum()
    total_engagement = data['engagement'].sum()
    avg_engagement_rate = total_engagement / total_posts if total_posts > 0 else 0
    
    # Platform performance
    top_platform = data.groupby('platform')['engagement'].sum().idxmax()
    top_platform_engagement = data.groupby('platform')['engagement'].sum().max()
    
    # Time analysis
    date_range = (data['fecha'].max() - data['fecha'].min()).days
    daily_avg_posts = total_posts / date_range if date_range > 0 else total_posts
    
    # Peak day
    daily_totals = data.groupby('fecha')['engagement'].sum()
    peak_day = daily_totals.idxmax()
    peak_engagement = daily_totals.max()
    
    summary = f"""
    ## 📊 Executive Summary - Social Listening Analytics
    
    ### Key Performance Indicators
    - **Total Posts Analyzed**: {total_posts:,}
    - **Total Engagement**: {total_engagement:,}
    - **Average Engagement per Post**: {avg_engagement_rate:.2f}
    - **Analysis Period**: {date_range} days
    - **Daily Average Posts**: {daily_avg_posts:.1f}
    
    ### Platform Performance
    - **Top Performing Platform**: {top_platform}
    - **Top Platform Total Engagement**: {top_platform_engagement:,}
    
    ### Peak Performance
    - **Highest Engagement Day**: {peak_day.strftime('%Y-%m-%d')}
    - **Peak Day Engagement**: {peak_engagement:,}
    
    ### Platform Distribution
    """
    
    platform_stats = data.groupby('platform').agg({
        'posts': 'sum',
        'engagement': 'sum'
    }).reset_index()
    
    for _, row in platform_stats.iterrows():
        percentage = (row['posts'] / total_posts) * 100
        avg_eng = row['engagement'] / row['posts'] if row['posts'] > 0 else 0
        summary += f"\n    - **{row['platform']}**: {row['posts']:,} posts ({percentage:.1f}%) | Avg Engagement: {avg_eng:.2f}"
    
    return summary

# Display executive summary
summary = generate_executive_summary(df)
print(summary)

## 8. Data Export & Sharing

In [ ]:
# Export visualizations and create shareable HTML report
def export_visualizations():
    """
    Exports key visualizations as HTML files for sharing
    """
    # Create exports directory
    export_dir = 'exports'
    os.makedirs(export_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Export dashboard
    dashboard_file = os.path.join(export_dir, f'social_listening_dashboard_{timestamp}.html')
    dashboard_fig.write_html(dashboard_file)
    
    # Export platform analysis
    platform_file = os.path.join(export_dir, f'platform_analysis_{timestamp}.html')
    platform_fig.write_html(platform_file)
    
    # Export engagement analysis
    engagement_file = os.path.join(export_dir, f'engagement_analysis_{timestamp}.html')
    engagement_fig.write_html(engagement_file)
    
    print(f"✓ Visualizations exported to {export_dir}/")
    print(f"  - Dashboard: {dashboard_file}")
    print(f"  - Platform Analysis: {platform_file}")
    print(f"  - Engagement Analysis: {engagement_file}")
    
    return export_dir

# Export visualizations
export_dir = export_visualizations()

print("\n🎯 Analysis Complete!")
print("All visualizations have been generated and are ready for analysis.")
print(f"Exported files can be found in the '{export_dir}' directory.")

## 9. Conclusion

This comprehensive analytics notebook provides multiple perspectives on your social listening data:

1. **Time Series Analysis** - Track evolution and trends over time
2. **Platform Comparison** - Compare performance across different social media platforms
3. **Engagement Analysis** - Deep dive into user interaction patterns
4. **Advanced Visualizations** - 3D plots and interactive charts for detailed exploration
5. **Executive Summary** - Key insights and performance indicators

### Next Steps:
- Use the exported HTML files for presentations and reports
- Modify date ranges and filters to analyze specific time periods
- Add additional metrics or dimensions based on your specific needs
- Set up automated reporting by scheduling this notebook to run regularly

### Technical Notes:
- All visualizations are interactive and can be explored in detail
- Charts are optimized for both desktop and mobile viewing
- Color schemes and styling follow modern data visualization best practices
- Export functionality allows for easy sharing and integration into presentations